# Data treatment

## 0. Install and import Dependencies 

In [2]:
#!pip install mediapipe opencv-python

In [1]:
import mediapipe as mp
import cv2
import pandas as pd
import numpy as np

In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

## 1. Make Detections from Feed

In [7]:
cap = cv2.VideoCapture(0)

# initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor Feed:
        image = cv2.cvtColor(cv2.flip(frame,1), cv2.COLOR_BGR2RGB)
        # Make Detections
        results = holistic.process(image)

        # Recolor image back to BGR for rendering
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # # pose 
        # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

        # right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        # left hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)


        cv2.imshow('Hand Model Detections', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [3]:
data = pd.read_csv('train_labels.csv')
data.head(5)

,filename,width,height,class,xmin,ymin,xmax,ymax
0,00002.png,640,480,c,286,158,420,328
1,00003.png,640,480,y,275,229,432,424
2,00004.png,640,480,y,167,161,382,380
3,00005.png,640,480,z,312,232,453,431
4,00006.png,640,480,z,289,175,415,391


In [18]:
data

,filename,width,height,class,xmin,ymin,xmax,ymax
0,00002.png,640,480,c,286,158,420,328
1,00003.png,640,480,y,275,229,432,424
2,00004.png,640,480,y,167,161,382,380
3,00005.png,640,480,z,312,232,453,431
4,00006.png,640,480,z,289,175,415,391
...,...,...,...,...,...,...,...,...
1689,z_5.png,640,480,z,411,127,531,338
1690,z_6.png,640,480,z,519,143,640,361
1691,z_7.png,640,480,z,308,339,470,480
1692,z_8.png,640,480,z,544,303,640,480


In [4]:
filenames = data['filename'].values
filenames

array(['00002.png', '00003.png', '00004.png', ..., 'z_7.png', 'z_8.png',
       'z_9.png'], dtype=object)

In [5]:
%cd images/train

c:\Users\utilisateur\Desktop\brief\youssef\sign_detection\images\train


### 'labelisation' des images avec Mediapipe

In [6]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
megaListe=[]
filename_final=[]
# For static images:
with mp_holistic.Holistic(static_image_mode=True) as holistic:
  for idx, file in enumerate(filenames):
    image = cv2.imread(file)
    image_height, image_width, _ = image.shape
    # Convert the BGR image to RGB before processing.
    results = holistic.process(image)

    if results.pose_landmarks:
      
        hand_landmarks= results.right_hand_landmarks
        prev=""
        liste=[]
        for letter in (str(hand_landmarks)):
            try:
                number=int(letter)
                prev+=letter
            except:
                if letter=="-" or letter =='.' or letter =='e':
                    prev+=letter
                elif prev != "" and len(prev)>2:
                    liste.append(float(prev))
                    prev=""
    if len(liste)>0:
        print(file)
        filename_final.append(file)
        megaListe.append(liste)

00119.png
00120.png
00121.png
00122.png
00123.png
00124.png
00125.png
00126.png
00127.png
00167.png
00168.png
00169.png
00170.png
00171.png
00172.png
00173.png
00174.png
00175.png
00189.png
00190.png
00191.png
00192.png
00200.png
00207.png
00247.png
00272.png
00274.png
00279.png
00283.png
00284.png
00285.png
00286.png
A17.jpg
A4.jpg
A5.jpg
A7.jpg
a_9.png
B0.jpg
B1.jpg
B3.jpg
B6.jpg
B7.jpg
B8.jpg
b_2.png
b_25.png
b_26.png
b_27.png
b_28.png
C0.jpg
C1.jpg
C2.jpg
C3.jpg
C5.jpg
C7.jpg
C8.jpg
C9.jpg
D0.jpg
D1.jpg
D2.jpg
D6.jpg
d_6.png
E0.jpg
E1.jpg
E2.jpg
E5.jpg
E6.jpg
E9.jpg
e_1.png
e_2.png
e_25.png
F1.jpg
F2.jpg
F3.jpg
F6.jpg
F7.jpg
frame_10056.jpg
frame_10067.jpg
frame_10082.jpg
frame_10474.jpg
frame_10477.jpg
frame_10480.jpg
frame_10483.jpg
frame_10486.jpg
frame_10489.jpg
frame_10492.jpg
frame_10495.jpg
frame_10590.jpg
frame_10594.jpg
frame_10601.jpg
frame_10608.jpg
frame_10644.jpg
frame_10882.jpg
frame_10899.jpg
frame_11045.jpg
frame_11049.jpg
frame_11053.jpg
frame_11057.jpg
frame_11061

In [9]:
# création d'un dataset avec les coordonnées x, y, z pour chaque keypoint 
dataLabels=pd.DataFrame(megaListe)
dataLabels.head()

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
0,0.233467,0.877692,-0.00002,0.276154,0.901083,-0.037816,0.295906,0.937619,-0.076339,0.300179,...,-0.101688,0.178426,0.995133,-0.126697,0.175585,1.020865,-0.140277,0.176958,1.039768,-0.149001
1,0.233467,0.877692,-0.00002,0.276154,0.901083,-0.037816,0.295906,0.937619,-0.076339,0.300179,...,-0.101688,0.178426,0.995133,-0.126697,0.175585,1.020865,-0.140277,0.176958,1.039768,-0.149001
2,0.233467,0.877692,-0.00002,0.276154,0.901083,-0.037816,0.295906,0.937619,-0.076339,0.300179,...,-0.101688,0.178426,0.995133,-0.126697,0.175585,1.020865,-0.140277,0.176958,1.039768,-0.149001
3,0.233467,0.877692,-0.00002,0.276154,0.901083,-0.037816,0.295906,0.937619,-0.076339,0.300179,...,-0.101688,0.178426,0.995133,-0.126697,0.175585,1.020865,-0.140277,0.176958,1.039768,-0.149001
4,0.233467,0.877692,-0.00002,0.276154,0.901083,-0.037816,0.295906,0.937619,-0.076339,0.300179,...,-0.101688,0.178426,0.995133,-0.126697,0.175585,1.020865,-0.140277,0.176958,1.039768,-0.149001


In [13]:
classe=data['class'].values
indexes=[]
for element in filename_final:
    indexes.append(np.where(filenames==element))
classe=classe[indexes]
classlist=[]
for element in classe:
    classlist.append(element[0][0])

dataLabels['classe']=classlist

In [16]:
dataLabels.shape

(794, 64)

In [17]:
dataLabels.to_csv('labellised_images.csv')